In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 3})
sns.set_style('whitegrid',
              {'grid.linestyle': ':', 'grid.color': 'red', 'axes.edgecolor': '0.5',
               'axes.linewidth': 1.2, 'legend.frameon': True})

import os
import pickle

import time

In [2]:
from scipy.constants import e, m_p, c

from scipy.constants import physical_constants

In [3]:
from cpymad.madx import Madx

import sixtracklib as pyst
import pysixtrack

In [4]:
import sys
sys.path.append('/home/oeftiger/gsi/git/python3/PyHEADTAIL/')
from PyHEADTAIL.particles import generators

PyHEADTAIL v1.13.5




In [5]:
nmass = physical_constants['atomic mass constant energy equivalent in MeV'][0] * 1e-3
nmass = 0.931494061 # MAD-X value

In [6]:
tune_range_qx = np.arange(18.55, 18.95 + 0.01, 0.01)
tune_range_qy = tune_range_qx

# prepare MAD-X error sequences

In [7]:
# e_seed = 1

# for qx in tune_range_qx:
#     for qy in tune_range_qy:
#         qqx, qqy = int(np.round((qx%1) * 100)), int(np.round((qy%1) * 100))
        
#         filename_error_table = "./error_tables/errors_{qqx}_{qqy}_{eseed:d}".format(
#             qqx=qqx, qqy=qqy, eseed=e_seed)
        
#         if os.path.exists(filename_error_table):
#             continue

#         print ('\n\n\n=== Preparing for Qx = {:.2f} and Qy = {:.2f} ===\n\n\n'.format(qx, qy))

# PySTL: read in lattice with errors in full definition

In [8]:
nturns = 20000 #2**16
npart = 1000

A = 238
Q = 28

Ekin_per_nucleon = 0.2e9 # in eV

###

mass = A * nmass * 1e9 * e / c**2 # in kg
charge = Q * e # in Coul

Ekin = Ekin_per_nucleon * A
p0c = np.sqrt(Ekin**2 + 2*Ekin*mass/e * c**2) # in eV

Etot = np.sqrt(p0c**2 + (mass/e)**2 * c**4) * 1e-9 # in GeV
p0 = p0c / c * e # in SI units
gamma = np.sqrt(1 + (p0 / (mass * c))**2)
beta = np.sqrt(1 - gamma**-2)


In [9]:
epsx_rms_fin = 35e-6 / 4 # geometrical emittances
epsy_rms_fin = 15e-6 / 4

limit_n_rms_x = 2
limit_n_rms_y = 2
limit_n_rms_z = 3.4

sig_z = 58 / 4. # in m
sig_dp = 0.5e-3

###

epsx_gauss = epsx_rms_fin * 1.778
epsy_gauss = epsy_rms_fin * 1.82

epsn_x = epsx_gauss * beta * gamma
epsn_y = epsy_gauss * beta * gamma

beta_z = sig_z / sig_dp

In [10]:
e_seed = 1

qx = tune_range_qx[0]
qy = tune_range_qy[0]

qqx, qqy = int(np.round((qx%1) * 100)), int(np.round((qy%1) * 100))

filename_error_table = "./error_tables/errors_{qqx}_{qqy}_{eseed:d}".format(
            qqx=qqx, qqy=qqy, eseed=e_seed)

madx = Madx()
madx.options.echo = False
madx.options.warn = False

madx.call('./SIS100_RF_220618_9slices.thin.seq')

madx.command.beam(particle='ion', mass=A*nmass, charge=Q, energy=Etot)

madx.call('OpticsYEH_BeamParameters.str')
madx.call('Coll+Errors+BeamDistr.madx')

madx.input('''
select, flag=seqedit, class=collimator;
select, flag=seqedit, class=kicker;
select, flag=seqedit, class=tkicker;
select, flag=seqedit, class=elseparator;

seqedit, sequence=SIS100RING;
    remove, element=selected;
    flatten;
endedit;

select, flag=seqedit, class=marker;
seqedit, sequence=SIS100RING;
    remove, element=selected;
    install, element=SIS100RING$START, s=0;
    flatten;
endedit;
''')

madx.use(sequence='sis100ring')

### --> first match, then add errors, then TWISS!

madx.input('''
    match, sequence=SIS100RING;
    global, sequence=SIS100RING, q1={qx}, q2={qy};
    vary, name=kqf, step=0.00001;
    vary, name=kqd, step=0.00001;
    lmdif, calls=500, tolerance=1.0e-10;
    endmatch;
'''.format(qx=qx, qy=qy)
)

madx.command.eoption(add=True, seed=1)
madx.command.exec('EA_EFCOMP_MH()')
for s in range(1, 10):
    assert madx.command.exec(f'EA_rEFCOMP_QD({s},1)')

twiss = madx.twiss();

madx.command.select(flag='error', pattern='QD11..', class_='MULTIPOLE')
madx.command.select(flag='error', pattern='QD12..', class_='MULTIPOLE')
madx.command.select(flag='error', pattern='mh1', class_='MULTIPOLE')
madx.command.select(flag='error', pattern='mh2', class_='MULTIPOLE')
madx.command.esave(file=filename_error_table)

madx.input('cavity_voltage = 58.2/1000/number_cavities;')

madx.command.readtable(file=filename_error_table, table="errors")
errors = madx.table.errors

sis100 = madx.sequence.sis100ring

# particle initialisation from pyheadtail

D_x_0 = twiss['dx'][0] * beta
D_y_0 = twiss['dy'][0] * beta

np.random.seed(0)

pyht_beam = generators.generate_Gaussian6DTwiss(
    npart, 1, charge, mass, twiss['s'][-1], gamma,
    twiss['alfx'][0], twiss['alfy'][0], twiss['betx'][0], twiss['bety'][0],
    1, epsn_x, epsn_y, 1,
    dispersion_x=D_x_0 if D_x_0 else None,
    dispersion_y=D_y_0 if D_y_0 else None,
    limit_n_rms_x=limit_n_rms_x**2, limit_n_rms_y=limit_n_rms_y**2, 
    limit_n_rms_z=limit_n_rms_z**2,
)

distribution_z_uncut = generators.gaussian2D(
    sig_z**2)
is_accepted = generators.make_is_accepted_within_n_sigma(
    epsn_rms=sig_z,
    limit_n_rms=limit_n_rms_z,
)
distribution_z_cut = generators.cut_distribution(distribution_z_uncut, is_accepted)

z, dp = distribution_z_cut(npart)
pyht_beam.z, pyht_beam.dp = z, dp / beta_z

### PySixTrack, lattice transfer and preparation!

pysixtrack_elements, _ = pysixtrack.Line.from_madx_sequence(
    sis100, exact_drift=True, install_apertures=True
)

pysixtrack_elements.remove_zero_length_drifts(inplace=True);
pysixtrack_elements.merge_consecutive_drifts(inplace=True);

 # add alignment and multipole errors

pysixtrack_elements.apply_madx_errors(error_table=errors)

### Load lattice into SixTrackLib

elements = pyst.Elements.from_line(pysixtrack_elements)
elements.BeamMonitor(num_stores=nturns);

### Transfer particles into SixTrackLib

particles = pyst.Particles.from_ref(npart, p0c=p0c, mass0=A*nmass*1e9, q0=Q, state=1)

particles.x[:] = pyht_beam.x
particles.px[:] = pyht_beam.xp
particles.y[:] = pyht_beam.y
particles.py[:] = pyht_beam.yp
particles.zeta[:] = pyht_beam.z
particles.delta[:] = pyht_beam.dp

particles.rpp[:] = 1. / (pyht_beam.dp + 1)

restmass = mass * c**2
restmass_sq = restmass**2
E0 = np.sqrt((p0 * c)**2 + restmass_sq)
p = p0 * (1 + pyht_beam.dp)
E = np.sqrt((p * c)**2 + restmass_sq)
particles.psigma[:] = (E - E0) / (beta * p0 * c)

gammai = E / restmass
betai = np.sqrt(1 - 1. / (gammai * gammai))
particles.rvv[:] = betai / beta

### prepare trackjob in SixTrackLib

trackjob = pyst.CudaTrackJob(elements, particles)

print ('\n\n\n' + '+'*26 + '\n*** ready for tracking ***\n' + '+'*26 + '\n')


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2019.10.28 11:44:02      +
  ++++++++++++++++++++++++++++++++++++++++++++
START MATCHING

number of sequences: 1
sequence name: sis100ring
number of variables:    2
user given constraints: 2
total constraints:      2

START LMDIF:


MATCH SUMMARY

Node_Name                  Constraint   Type  Target Value       Final Value        Penalty
--------------------------------------------------------------------------------------------------


Final Penalty Function =   2.51160496e-15





Variable                 Final Value  Initial Value Lower Limit  Upper Limit 
--------------------------------------------------------------------------------
kqf                       2.77739e-01  2.81571e-01 -1.00000e+20  1.00000e+20
kqd                      -2.77252e-01 -2.80616e-01 -1.00000e+20  1.

In [11]:
t0 = time.time()

trackjob.track_until(nturns)
trackjob.collect()

store = {}
filename_error_table = os.path.basename(filename_error_table)

# statistics
x = trackjob.output.particles[0].x.reshape((nturns, npart)).T
store['std_x'] = np.mean(np.std(x, axis=0)[-50:])
y = trackjob.output.particles[0].y.reshape((nturns, npart)).T
store['std_y'] = np.mean(np.std(y, axis=0)[-50:])

# losses
pbuffer = trackjob.particles_buffer.get_object(0)
np.save('results/' + filename_error_table + '_alive.npy', pbuffer.state)
np.save('results/' + filename_error_table + '_lost_at_element.npy', 
        pbuffer.at_element[~pbuffer.state.astype(bool)])
np.save('results/' + filename_error_table + '_lost_at_turn.npy',
        pbuffer.at_turn[~pbuffer.state.astype(bool)])

store['losses'] = np.sum(pbuffer.state)

# finish job
pickle.dump(store, open('results/' + filename_error_table + '_summary.p', 'wb'))

!touch "results/$filename_error_table"_done

del trackjob, elements, particles

t1 = time.time()

In [12]:
(t1 - t0) / 60.

33.21658913691839

In [14]:
np.sum(pbuffer.state)

712

In [41]:
!ls ./Vera_1match-2err_55_55/

1match-2err.zip  track_55_55_1_20000_one  tracksumm_55_55_1_20000
errors_55_55_1	 trackloss_55_55_1_2000   twiss_55_55_1
matched_55_55_1  tracksumm_1.madx


In [52]:
np.genfromtxt('./Vera_1match-2err_55_55/tracksumm_55_55_1_20000', skip_header=8)[:1000]

array([[ 1.00000000e+00,  0.00000000e+00, -1.49495704e-02, ...,
         2.86873274e-04,  0.00000000e+00,  0.00000000e+00],
       [ 2.00000000e+00,  0.00000000e+00, -1.56963544e-02, ...,
         2.83204003e-04,  0.00000000e+00,  0.00000000e+00],
       [ 3.00000000e+00,  0.00000000e+00,  2.21776800e-02, ...,
         3.34435334e-05,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 9.98000000e+02,  0.00000000e+00,  1.80188790e-02, ...,
         2.13486304e-04,  0.00000000e+00,  0.00000000e+00],
       [ 9.99000000e+02,  0.00000000e+00, -1.52037930e-02, ...,
         8.33387393e-05,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+03,  0.00000000e+00, -1.04522653e-02, ...,
        -2.65525862e-04,  0.00000000e+00,  0.00000000e+00]])

In [55]:
!head -7 ./Vera_1match-2err_55_55/tracksumm_55_55_1_20000 | tail -1

*     NUMBER       TURN                  X                 PX                  Y                 PY                  T                 PT                  S                  E 


In [45]:
np.genfromtxt('./Vera_Tunes_Emittances.dat')

array([[           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [5.50000000e+01, 5.50000000e+01, 1.00000000e+00, ...,
        5.10266004e-06, 5.10787475e-06, 7.95000000e+02],
       [5.50000000e+01, 5.60000000e+01, 1.00000000e+00, ...,
        8.18066600e-06, 4.36666198e-06, 9.86000000e+02],
       ...,
       [9.50000000e+01, 9.30000000e+01, 1.00000000e+00, ...,
        1.57863354e-05, 3.99121279e-06, 8.97000000e+02],
       [9.50000000e+01, 9.40000000e+01, 1.00000000e+00, ...,
        1.33605337e-05, 3.83190357e-06, 6.96000000e+02],
       [9.50000000e+01, 9.50000000e+01, 1.00000000e+00, ...,
        6.88273417e-06, 4.73707620e-06, 3.33000000e+02]])

In [56]:
# e_seed = 1

# for qx in tune_range_qx:
#     for qy in tune_range_qy:
#         qqx, qqy = int(np.round((qx%1) * 100)), int(np.round((qy%1) * 100))
        
#         filename_error_table = "./errors_{qqx}_{qqy}_{eseed:d}".format(
#             qqx=qqx, qqy=qqy, eseed=e_seed)
        
#         if os.path.exists(filename_error_table + '_done'):
#             continue
        
#         print ('\n\n\n=== Running at Qx = {:.2f} and Qy = {:.2f} ===\n\n\n'.format(qx, qy))
        
#         # read in MAD-X sequence with errors
        
#         madx = Madx()
#         madx.options.echo = False
#         madx.options.warn = False
#         madx.options.info = False
        
#         madx.call(file=filename_error_table + '_full-lattice.seq')
        
#         madx.command.beam(particle='ion', mass=238*nmass, charge=28, energy=238*0.2+238*nmass)
        
#         madx.use(sequence='SIS100RING')
        
#         twiss = madx.twiss();
        
#         sis100 = madx.sequence.sis100ring
        
#         assert 2 == len(
#             [f.name for f in map(lambda x: x.base_type, sis100.expanded_elements) 
#              if 'drift' != f.name and 'multipole' != f.name and 'rfcavity' != f.name])
        
#         # go to PySTL
        
#         elements = pyst.Elements.from_mad(sis100)

#         elements.BeamMonitor(num_stores=nturns);

#         particles = pyst.Particles.from_ref(npart, p0c=p0c, q0=28, mass0=238*nmass*1e9)
        
#         # matching

#         np.random.seed(987654321)

#         x = np.random.normal(
#             loc=0, scale=np.sqrt(twiss['betx'][0] * epsx_gauss), size=npart)
#         xp = np.random.normal(
#              loc=0, scale=np.sqrt(epsx_gauss / twiss['betx'][0]), size=npart)
#         xp += -twiss['alfx'][0] / twiss['betx'][0] * x

#         y = np.random.normal(
#             loc=0, scale=np.sqrt(twiss['bety'][0] * epsy_gauss), size=npart)
#         yp = np.random.normal(
#              loc=0, scale=np.sqrt(epsy_gauss / twiss['bety'][0]), size=npart)
#         yp += -twiss['alfy'][0] / twiss['bety'][0] * y

#         z = np.random.normal(loc=0, scale=sig_z, size=npart)
#         dp = np.random.normal(loc=0, scale=sig_dp, size=npart)

#         particles.x *= 0
#         particles.px *= 0
#         particles.y *= 0
#         particles.py *= 0
#         particles.zeta *= 0
#         particles.delta *= 0

#         particles.x += x
#         particles.px += xp
#         particles.y += y
#         particles.py += yp
#         particles.zeta += z
#         particles.delta += dp
        
#         # PySTL job
        
#         job = pyst.TrackJob(elements, particles, device=pystl_device)

#         job.track(nturns)

#         job.collect()
        
#         # coordinates
# #         np.save(filename_error_table + '_x.npy', job.output.particles[0].x)
# #         np.save(filename_error_table + '_y.npy', job.output.particles[0].y)
# #         np.save(filename_error_table + '_px.npy', job.output.particles[0].px)
# #         np.save(filename_error_table + '_py.npy', job.output.particles[0].py)
# #         np.save(filename_error_table + '_zeta.npy', job.output.particles[0].zeta)
# #         np.save(filename_error_table + '_delta.npy', job.output.particles[0].delta)

#         store = {}

#         # statistics
#         x = job.output.particles[0].x.reshape((nturns, npart)).T
#         store['std_x'] = np.mean(np.std(x, axis=0)[-50:])
#         y = job.output.particles[0].y.reshape((nturns, npart)).T
#         store['std_y'] = np.mean(np.std(y, axis=0)[-50:])
        
#         # losses
#         pbuffer = job.particles_buffer.get_object(0)
#         np.save(filename_error_table + '_alive.npy', pbuffer.state)
#         np.save(filename_error_table + '_lost_at_element.npy', 
#                 pbuffer.at_element[~pbuffer.state.astype(bool)])
#         np.save(filename_error_table + '_lost_at_turn.npy',
#                 pbuffer.at_turn[~pbuffer.state.astype(bool)])
        
#         store['losses'] = np.sum(pbuffer.state)
        
#         # finish job
#         pickle.dump(store, open(filename_error_table + '_summary.p', 'wb'))

#         !touch "$filename_error_table"_done
        
#         del job, elements, particles